In [0]:
%sql
create or replace temp view item_count as (
select b.week_end_dt
        ,trim(a.stor_id) stor_id
        ,case when trim(a.stor_id) like 'GM%' then 'PFS' else 'CF' end as type
        ,sum(case when a.whse_it_dlv_qy>0 then a.tot_each_qy-a.tot_orig_oos_inv_qy-a.tot_orig_oos_pick_qy+a.tot_sub_ship_qy else 0 end) items_ship
        ,count(distinct case when a.whse_it_dlv_qy>0 then a.ord_id else null end) ords_ship
from merchandising.pos.rdt_ord_sum_cdm a
join merchandising.pos.rdt_tm_look_sdm b on a.dlv_dt_src=b.the_date
where it_dmnd_qy>0
and week_end_dt >= '2022-01-01' and a.dlv_dt_src < date_add(current_date, -pmod(dayofweek(current_date) - 1, 7))
group by 1,2,3)

In [0]:
%sql
create or replace temp view pfs_pickrate as (
with location as (select distinct StoreId, 
case when P3StoreId is null or P3StoreId = '' then StoreId else P3StoreId end as 
clean_store, 
case when StoreId like 'GM%' then 'PFS' else 'CF' end as type 
from managed_assets.location.storepickuppointmaster_cdm)
select b.storeId stor_id,
       c.week_end_dt,
       b.type,
       sum(a.cnt1_qy) unitpick,
       sum(timestampdiff(second, a.beg_dt_tm, a.end_dt_tm)) pickingseconds 
from merchandising.pos.emp_perf_ahold_sdm a left join location b on a.dc_id = b.clean_store
left join merchandising.pos.rdt_tm_look_sdm c on date(a.beg_dt_tm) = c.the_date
where a.dc_id != 'GKK' and a.task_id = 'P' and a.arg2_cd not in ('R','repick')
group by all)

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW gkk AS (
select dc_id as stor_id
, week_end_Dt
,'CF'as type
, units_pck unitpick
, shftsecs pickingseconds
from (select a.week_end_Dt
        , a.dc_id
        , (a.shftsecs+b.shftsecs) as shftsecs
        , (a.unitpick + b.units_picked) units_pck
        , a.unitpick as unitspickm
        , b.units_picked as unitspicka
from (select dc_id
        , week_end_dt
        , unitpick
        , shftsecs
from (  select dc_id
                ,week_end_dt
                ,sum(units_pck) unitpick
                ,sum(shft_secs) shftsecs
        from (  select distinct x1.dc_id
                        ,x1.ee_id
                        ,date(beg_dt_tm) as actv_dt
                        -- ,x2.pick_desc_tx
                        ,count(perf_id) picks
                        ,sum(cnt1_qy) units_pck
                        ,CAST(SUM(unix_timestamp(end_dt_tm) - unix_timestamp(beg_dt_tm)) AS INT) AS shft_secs
                from (  select distinct perf_id
                                , ee_id
                                ,dc_id
                                ,task_id
                                ,arg1_cd
                                ,arg2_cd
                                ,cnt1_qy
                                ,cnt2_qy
                                ,dlv_dt
                                ,beg_dt_tm
                                ,end_dt_tm
                                ,rec_stat_cd,xfer_cd 
                        from merchandising.pos.p3_amt_emp_perf_sdm 
                        where task_id = 'J' ) x1
                -- left join p3_daily_curr:cmt_area_ctl x2 on x1.dc_id = x2.dc_id and x1.arg2_Cd = x2.pick_Area_cd
                where x1.dc_id = 'GKK'
                group by 1,2,3) a 
        inner join merchandising.pos.rdt_tm_look_sdm b on a.actv_Dt = b.the_Date
        where b.week_end_dt >= '2022-01-01'
        group by 1,2)final_pick 
group by 1,2,3,4) a
join (select week_end_dt
        , dc_id
        , sum(unitspicked) units_picked
        , round(sum(seconds),2) shftsecs
        from (
                select week_end_dt
                , dc_id
                , sum(TRANSQTY) unitspicked
                , SUM(ABS(CAST(unix_timestamp(endtdttm) - unix_timestamp(startdttm) AS BIGINT))) AS seconds
                -- , sum(ABS(((endtdttm - startdttm)::INTERVAL second(8) to second)::CHAR(10)::INT8)) seconds
                from (
                        select week_end_dt 
                        , dc_id
                        , m1.updated_by
                        , m1.To_TU
                        , m1.autostore_grid
                        , sum(m1.TO_QUANTITY) TOQTYTOTAL
                        , sum(m1.TRANSACTION_COUNT) TRANSCOUNT
                        , sum(m1.Transaction_Quantity) TRANSQTY
                        , min(m1.CREATE_DATE) startDtTm 
                        , max(m1.CREATE_DATE) EndtDtTm
                        , SUM(ABS(CAST((unix_timestamp(m1.EndDateTime) - unix_timestamp(m1.create_date)) / 60 AS BIGINT))) AS Minutes
                        from (
                                select t.week_end_dt
                                , x1.updated_by
                                , x1.from_location as autostore_grid
                                , case when x1.WarehouseID = 'GKK' then 'GKK' else 'GKK' end as dc_id
                                , x1.from_workstation
                                , x1.TO_QUANTITY
                                , x1.TRANSACTION_COUNT
                                , x1.Transaction_Quantity
                                , x1.To_TU
                                , x1.CREATE_DATE
                                , lead(x1.CREATE_DATE) over (partition by date(x1.CREATE_DATE),x1.From_Workstation, x1.UPDATED_BY order by x1.CREATE_DATE) EndDateTime
                                from  merchandising.pos.centralfulfillmenttransactions_cdm x1
                                join merchandising.pos.rdt_tm_look_sdm t on date(x1.CREATE_DATE) = t.the_date
                                where t.week_end_dt >= '2022-01-01'
                                and x1.transaction_type = 'PICK'
                                ) M1
                                group by 1,2,3,4,5) M2
                group by 1,2) M3
group by 1,2) b on a.dc_id = b.dc_id and a.week_end_Dt = b.week_end_Dt))

In [0]:
%sql
create or replace temp view cf_pickrate_2 as (
SELECT dc_id as stor_id
        , date(week_end_dt) week_end_dt
        ,'CF' as type
        , unitpick
        , shftsecs as pickingseconds
        FROM (  select dc_id
                ,week_end_Dt
                ,sum(units_pck) unitpick
                ,sum(shft_secs) shftsecs 
        from (  select distinct x1.dc_id
                        ,x1.ee_id
                        ,date(beg_dt_tm) as actv_dt
                        -- ,x2.pick_desc_tx
                        ,count(perf_id) picks
                        ,sum(cnt1_qy) units_pck
                        ,CAST(SUM(unix_timestamp(end_dt_tm) - unix_timestamp(beg_dt_tm)) AS INT) AS shft_secs
                from (  select distinct perf_id
                                ,ee_id
                                ,dc_id
                                ,task_id
                                ,arg1_cd
                                ,arg2_cd
                                ,cnt1_qy
                                ,cnt2_qy
                                ,dlv_dt
                                ,beg_dt_tm
                                ,end_dt_tm
                                ,rec_stat_cd,xfer_cd 
                        from merchandising.pos.p3_amt_emp_perf_sdm
                        where task_id = 'P' ) x1  
                where x1.dc_id like 'G%' and x1.dc_id not in ('GKK')
                group by 1,2,3) a 
        inner join merchandising.pos.rdt_tm_look_sdm b on a.actv_Dt = b.the_Date 
        where week_end_dt >= '2022-01-01'
        group by 1,2) FINAL_PICK 
group by 1,2,3,4,5)

In [0]:
%sql
create or replace temp view 2022_data as (
  with union_table as (select dc_id, 
                           task_id,
                           arg2_cd, 
                           beg_dt_tm, end_dt_tm,
                           cnt1_qy
 from merchandising.pos.emp_perf_ahold_sdm union select dc_id, 
                           task_id,
                           arg2_cd, 
                           beg_dt_tm, end_dt_tm, cnt1_qy from merchandising.pos.p3_amt_emp_perf_sdm),
location as (select distinct StoreId, 
case when P3StoreId is null or P3StoreId = '' then StoreId else P3StoreId end as 
clean_store, 
case when StoreId like 'GM%' then 'PFS' else 'CF' end as type 
from managed_assets.location.storepickuppointmaster_cdm)
select b.storeId stor_id,
       c.week_end_dt,
       b.type,
       sum(a.cnt1_qy) unitpick,
       sum(timestampdiff(second, a.beg_dt_tm, a.end_dt_tm)) pickingseconds 
from union_table a left join location b on a.dc_id = b.clean_store
left join merchandising.pos.rdt_tm_look_sdm c on date(a.beg_dt_tm) = c.the_date
where a.dc_id != 'GKK' and a.task_id = 'P' and a.arg2_cd not in ('R','repick') and c.week_end_dt <= '2022-12-31'
group by all
)

In [0]:
%sql
create or replace temp view all_pickrate as (
  select stor_id,
  week_end_dt,
  type,
  sum(unitpick) unitpick,
  sum(pickingseconds) pickingseconds from 
  (select * from pfs_pickrate
  union
  select * from cf_pickrate_2
  union
  select * from gkk)
  group by all
)

In [0]:
%sql
create or replace temp view pickrate as (
with location as (select distinct StoreId, 
case when P3StoreId is null or P3StoreId = '' then StoreId else P3StoreId end as 
clean_store, 
case when StoreId like 'GM%' then 'PFS' else 'CF' end as type 
from managed_assets.location.storepickuppointmaster_cdm),
last as (select b.storeId stor_id,
       c.week_end_dt,
       b.type,
       sum(a.count1quantity) unitpick,
       sum(timestampdiff(second, a.begindatetime, a.enddatetime)) pickingseconds 
from merchandising.pos.employeeorderitemactivitytracker_cdm a left join location b on a.storeId = b.clean_store
left join merchandising.pos.rdt_tm_look_sdm c on date(a.begindatetime) = c.the_date
where a.storeId != 'GKK' and a.taskid = 'P' and a.Argument2Code not in ('R','repick') and c.week_end_dt >= '2023-01-07'
group by all)
select * from last
union
select * from gkk
union
select * from 2022_data)

In [0]:
%sql
create or replace table `test-analytics`.custom_datamart.productivity_data_load as (
select int(e.ahdz_per), 
       int(e.ahdz_week),
       int(e.ahdz_qtr),
       cast(left(e.ahdz_week,4) as int) as year,
       f.banner, f.type as t2, f.open_closed, f.date_open, f.date_closed, 
        a.*,
       d.unitpick, 
       d.pickingseconds
from item_count a
left join pickrate d on a.week_end_dt = d.week_end_dt and a.stor_id = d.stor_id and a.type = d.type
left join (select distinct week_end_dt, ahdz_per, ahdz_week, ahdz_qtr from merchandising.pos.rdt_tm_look_sdm) e on a.week_end_dt = e.week_end_dt
left join `test-analytics`.custom_datamart.cf_list_db_upload f on a.stor_id = f.store_id
)